In [1]:
%load_ext tensorboard

from concurrent.futures import ProcessPoolExecutor, as_completed
from datetime import datetime
import pathlib
import os
import gc
from core import Model
import numpy as np
import math

os.environ['CUDA_​DEVICE_​ORDER'] = 'PCI_BUS_ID'
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'
clock_start = datetime.now().strftime("%Y%m%d-%H%M%S")
#clock_start = "20240722-085506"
workdir = str(pathlib.Path().resolve())


n = 4
perms = np.empty((math.factorial(n), n), dtype=np.uint8, order='F')
perms[0, 0] = 0

rows_to_copy = 1
for i in range(1, n):
    perms[:rows_to_copy, i] = i
    for j in range(1, i + 1):
        start_row = rows_to_copy * j
        end_row = rows_to_copy * (j + 1)
        splitter = i - j
        perms[start_row: end_row, splitter] = i
        perms[start_row: end_row, :splitter] = perms[:rows_to_copy, :splitter]
        perms[start_row: end_row, splitter + 1:i + 1] = perms[:rows_to_copy, splitter:i]

    rows_to_copy *= i + 1



2024-07-31 07:50:12.368338: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-31 07:50:13.015148: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:

def frontload_data():
    from tensorflow import keras
    mnist_dataset = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()
    mnist_dataset = keras.datasets.mnist
    (train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

def pool_job(input):
    this_model = Model(*input)
    this_model.set_data()
    #this_model.prep()
    this_model.prep_variant(input[-1])
    this_model.pre_filter()
    this_model.fit()
    this_model.save_filtered()

frontload_data()
#to prevent race condition
with ProcessPoolExecutor(8) as executor:
    runner = {
            executor.submit(pool_job, (1,2,clock_start,workdir,m)): m for m in perms
        }
    for future in as_completed(runner):
        runner.pop(future)






2024-07-31 07:50:15.109024: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:50:15.109088: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:50:15.109542: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:50:15.109731: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:50:15.110187: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:50:15.110364: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:50:15.110394: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:50:15.110397: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:50:15.110530: I external/

  3/118 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step

I0000 00:00:1722412219.775322   88146 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


  5/118 ━━━━━━━━━━━━━━━━━━━━ 9s 81ms/step

I0000 00:00:1722412220.011794   88179 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1722412220.022236   88115 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
I0000 00:00:1722412220.071062   88210 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


...

118/118 - 1s - 7ms/step - accuracy: 0.8601 - loss: 0.3976 - val_accuracy: 0.8362 - val_loss: 0.4609
Epoch 20/30
118/118 - 1s - 7ms/step - accuracy: 0.8620 - loss: 0.3894 - val_accuracy: 0.8411 - val_loss: 0.4516
118/118 - 1s - 7ms/step - accuracy: 0.8600 - loss: 0.3981 - val_accuracy: 0.8395 - val_loss: 0.4536
Epoch 23/30
Epoch 16/30
118/118 - 1s - 7ms/step - accuracy: 0.8535 - loss: 0.4165 - val_accuracy: 0.8299 - val_loss: 0.4734
Epoch 15/30
118/118 - 1s - 7ms/step - accuracy: 0.8674 - loss: 0.3779 - val_accuracy: 0.8428 - val_loss: 0.4382
Epoch 24/30
118/118 - 1s - 7ms/step - accuracy: 0.8637 - loss: 0.3881 - val_accuracy: 0.8438 - val_loss: 0.4441
Epoch 25/30
118/118 - 1s - 6ms/step - accuracy: 0.8610 - loss: 0.3955 - val_accuracy: 0.8405 - val_loss: 0.4525
Epoch 17/30
118/118 - 1s - 6ms/step - accuracy: 0.8678 - loss: 0.3766 - val_accuracy: 0.8431 - val_loss: 0.4379
Epoch 25/30
118/118 - 1s - 6ms/step - accuracy: 0.8641 - loss: 0.3869 - val_accuracy: 0.8439 - val_loss: 0.4438
118/

2024-07-31 07:55:52.819093: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:55:52.819144: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.


118/118 - 1s - 11ms/step - accuracy: 0.8653 - loss: 0.3835 - val_accuracy: 0.8428 - val_loss: 0.4479
Epoch 23/30
118/118 - 1s - 7ms/step - accuracy: 0.8656 - loss: 0.3821 - val_accuracy: 0.8443 - val_loss: 0.4425
118/118 - 1s - 6ms/step - accuracy: 0.8648 - loss: 0.3857 - val_accuracy: 0.8388 - val_loss: 0.4576
Epoch 28/30
118/118 - 1s - 6ms/step - accuracy: 0.8658 - loss: 0.3788 - val_accuracy: 0.8419 - val_loss: 0.4484
118/118 - 1s - 7ms/step - accuracy: 0.8534 - loss: 0.4209 - val_accuracy: 0.8380 - val_loss: 0.4580
Epoch 9/30


2024-07-31 07:55:53.663102: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:70] Profiler session collecting data.
2024-07-31 07:55:53.685513: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1364] CUPTI activity buffer flushed
2024-07-31 07:55:53.709372: I external/local_xla/xla/backends/profiler/gpu/cupti_collector.cc:540]  GpuTracer has collected 75 callback api events and 62 activity events. 
2024-07-31 07:55:53.713922: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-31 07:55:53.714170: I external/local_tsl/tsl/profiler/rpc/client/save_profile.cc:144] Collecting XSpace to repository: /workspaces/QML-QPF/multiprocessing/output/20240731-075014/runs/FASHIONMNIST/FASHIONMNISTqcnot3-2-1-0/plugins/profile/2024_07_31_07_55_53/1ba22281c77a.xplane.pb


118/118 - 1s - 7ms/step - accuracy: 0.8658 - loss: 0.3820 - val_accuracy: 0.8419 - val_loss: 0.4474
Epoch 24/30
118/118 - 1s - 10ms/step - accuracy: 0.8590 - loss: 0.4005 - val_accuracy: 0.8313 - val_loss: 0.4670
Epoch 23/30
118/118 - 1s - 6ms/step - accuracy: 0.8652 - loss: 0.3846 - val_accuracy: 0.8386 - val_loss: 0.4574
Epoch 29/30
118/118 - 1s - 6ms/step - accuracy: 0.8557 - loss: 0.4148 - val_accuracy: 0.8397 - val_loss: 0.4543
Epoch 10/30
118/118 - 1s - 6ms/step - accuracy: 0.8663 - loss: 0.3805 - val_accuracy: 0.8419 - val_loss: 0.4469
Epoch 25/30
118/118 - 1s - 6ms/step - accuracy: 0.8594 - loss: 0.3992 - val_accuracy: 0.8315 - val_loss: 0.4666
Epoch 24/30
118/118 - 1s - 6ms/step - accuracy: 0.8655 - loss: 0.3835 - val_accuracy: 0.8384 - val_loss: 0.4573
Epoch 30/30
118/118 - 2s - 19ms/step - accuracy: 0.6746 - loss: 0.9472 - val_accuracy: 0.7726 - val_loss: 0.6441
Epoch 2/30
118/118 - 1s - 6ms/step - accuracy: 0.8575 - loss: 0.4096 - val_accuracy: 0.8416 - val_loss: 0.4513
Epo

In [3]:

#{pool_job((0,2,clock_start,workdir,m)): m for m in [perms[0]]}

In [4]:

def pool_job2(detail):
    restored_model = Model(*detail)
    restored_model.set_data()
    #restored_model.prep()
    restored_model.prep_variant(detail[-1])
    restored_model.load_history()
    restored_model.visuals.circuit()
    restored_model.visuals.weights()
    restored_model.visuals.relative_weights()
    restored_model.visuals.bias()
    restored_model.visuals.relative_bias()
    restored_model.visuals.data_samples()
    restored_model.visuals.relative_samples()
    restored_model.visuals.filtering()


with ProcessPoolExecutor(8) as executor:
    runner = {
            executor.submit(pool_job2, (1,2,clock_start,workdir,m)): m for m in perms
        }
    for future in as_completed(runner):
        runner.pop(future) 

2024-07-31 07:56:15.168148: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:56:15.168318: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:56:15.168884: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:56:15.168948: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:56:15.169009: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:56:15.169200: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:56:15.169248: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 07:56:15.169427: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 07:56:15.169840: I external/local

  1/313 ━━━━━━━━━━━━━━━━━━━━ 1:34 304ms/step

I0000 00:00:1722412576.179143  125211 service.cc:145] XLA service 0x7d4a58003470 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722412576.179182  125211 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-07-31 07:56:16.187688: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-31 07:56:16.232920: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1722412576.250519  125285 service.cc:145] XLA service 0x7d4a4c014d80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722412576.250563  125285 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
I0000 00:00:1722412576.257136  125253 service.cc:145] XLA service 0x7d4a4c003d60 initialized for p

216/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step  

I0000 00:00:1722412576.405675  125317 service.cc:145] XLA service 0x7d4a58003f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722412576.405717  125317 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-07-31 07:56:16.413378: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-07-31 07:56:16.417724: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1722412576.421481  125348 service.cc:145] XLA service 0x7d4a4c014b80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722412576.421523  125348 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
I0000 00:00:1722412576.422153  125331 service.cc:145] XLA service 0x7d4a58003a10 initialized for p

...

/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_bias/FASHIONMNISTFASHIONMNISTqcnot3-1-2-0.gif
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/individual_samples/FASHIONMNISTFASHIONMNISTqcnot2-3-0-1.png
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_samples/FASHIONMNISTFASHIONMNISTqcnot2-3-0-1.png


/workspaces/QML-QPF/multiprocessing/visuals.py:74: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  figure, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 12))


/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_bias/FASHIONMNISTFASHIONMNISTqcnot3-2-0-1.gif
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/filter/FASHIONMNISTFASHIONMNISTqcnot2-3-0-1.gif


/workspaces/QML-QPF/multiprocessing/visuals.py:74: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  figure, ax = plt.subplots(nrows=1, ncols=1, figsize=(12, 12))


/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_bias/FASHIONMNISTFASHIONMNISTqcnot3-2-1-0.gif
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/individual_samples/FASHIONMNISTFASHIONMNISTqcnot2-3-1-0.png
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_samples/FASHIONMNISTFASHIONMNISTqcnot2-3-1-0.png
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/filter/FASHIONMNISTFASHIONMNISTqcnot2-3-1-0.gif
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/individual_samples/FASHIONMNISTFASHIONMNISTqcnot3-0-1-2.png
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/individual_samples/FASHIONMNISTFASHIONMNISTqcnot3-1-0-2.png
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_samples/FASHIONMNISTFASHIONMNISTqcnot3-1-0-2.png
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/relative_samples/FASHIONMNISTFASHIONMNISTqcnot3-0-1-2.png
/worksp

In [5]:

def UMAP_job(detail):
    restored_model = Model(*detail)
    restored_model.set_data()
    restored_model.prep()
    restored_model.load_history()
    restored_model.visuals.manifold_umap()

{UMAP_job((1,2,clock_start,workdir,m)): m for m in perms}

2024-07-31 08:10:47.530694: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 08:10:47.530746: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 08:10:47.551672: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1239] Profiler found 1 GPUs
2024-07-31 08:10:47.557477: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-31 08:10:47.557610: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1364] CUPTI activity buffer flushed
2024-07-31 08:10:47.953448: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:238] Using CUDA malloc Async allocator for GPU: 0
2024-07-31 08:10:47.953745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1928] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6795 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:b3:00.0, compute capability: 7.

  1/313 ━━━━━━━━━━━━━━━━━━━━ 1:25 273ms/step

I0000 00:00:1722413448.271162  130955 service.cc:145] XLA service 0x7d4a1c003890 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1722413448.271191  130955 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2080, Compute Capability 7.5
2024-07-31 08:10:48.278107: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-07-31 08:10:48.305147: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
I0000 00:00:1722413448.432857  130955 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested for previously compiled argument types ((uint32,)). This has no effect and perhaps indicates a bug in the calling code (compiling a ufunc more than once for the same signature
  warnings.warn(msg, errors.NumbaWarning)
/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/numba/np/ufunc/dufunc.py:343: NumbaWarning: Compilation requested fo

...

2024-07-31 08:17:57.067149: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:104] Profiler session initializing.
2024-07-31 08:17:57.067185: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:119] Profiler session started.
2024-07-31 08:17:57.087569: I external/local_tsl/tsl/profiler/lib/profiler_session.cc:131] Profiler session tear down.
2024-07-31 08:17:57.094617: I external/local_xla/xla/backends/profiler/gpu/cupti_tracer.cc:1364] CUPTI activity buffer flushed


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


/opt/conda/envs/QML-QPF/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


1875/1875 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step
/workspaces/QML-QPF/multiprocessing/output/20240731-075014/visuals/UMAP/FASHIONMNISTFASHIONMNISTqcnot3-2-1-0.html


{None: array([3, 2, 1, 0], dtype=uint8)}